In [4]:
import pandas as pd
import html
import re
import string

In [24]:
d = pd.read_csv("2018-E-c-En-train.txt",sep="\t")
d = d.drop(['ID'],axis=1)

emojis = {}
with open('data/emojilist5.csv', 'r') as f:
    for line in f:
        unic=line.split(',')[0].lower()
        trans=line.split(',')[1]
        emojis[unic]=trans
        
def emojify(text, emojis):
    text = str(text.encode('unicode-escape'))[2:-1]
    if '\\u' in text.lower():
        text = text.replace('\\\\U' , '\\\\u')
        text = text.replace('\\\\u' , ' \\\\u')
        words = text.split(' ')
        for word in words:
            if '\\u' in word:
                if word in emojis.keys():
                    words[words.index(word)] = emojis[word]
                elif word[0:11] in emojis:
                    word_1 = word[11:len(word)]
                    words[words.index(word)] = emojis[word[0:11]] + ' ' + word_1
                elif word[0:7] in emojis:
                    word_1 = word[7:len(word)]
                    words[words.index(word)] = emojis[word[0:7]] + ' ' + word_1
        return ' '.join(words)
    return text

def removeXA0(Tweet):
    return Tweet.replace(u'\xa0', u' ')
def removeLineFeed(Tweet):
    return Tweet.replace(u'\\n',u' ')
def removeCarriageReturn(Tweet):
    return Tweet.replace(u'\\r',u' ')
def removeCitation(Tweet):
    return re.sub(r"(?:@[A-Za-z0-9_]+)","",Tweet)
def removeAmp(Tweet):
    return Tweet.replace("&amp;","")
def removeSpecialCharacters(Tweet):
    whitelist = string.digits + "'" + string.ascii_letters + ' ' + '_'
    return ''.join(c for c in Tweet if c in whitelist)
def removeNumbersNotInsideCharacters(Tweet):
    return ' '.join(c for c in Tweet.split(" ") if not c.isnumeric())
def executePreProcessing():
    d.Tweet = d.Tweet.apply(removeXA0)
    d.Tweet = d.Tweet.apply(removeLineFeed)
    d.Tweet = d.Tweet.apply(removeCarriageReturn)
    d.Tweet = d.Tweet.apply(removeCitation)
    d.Tweet = d.Tweet.apply(removeAmp)
    d.Tweet = d.Tweet.apply(emojify, args=(emojis,))
    d.Tweet = d.Tweet.apply(removeSpecialCharacters)
    d.Tweet = d.Tweet.apply(removeNumbersNotInsideCharacters)
executePreProcessing()

Replace emojis with text translations given in emoji list (emojis: dictionary of translated emojis). Will probably add some extra slash characters so other processing steps can be done after this one. 

### There is no in the train dataset:
 - HTML tags
 - URLs
 - \t

In [28]:
d.iloc[259].Tweet

'india will take revenge  by making new movies like phantom and they will be like ghus_k_maren_ge this is how indian army take revenge'